# Bai 1

In [ ]:
# set eviroments
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!python -m bitsandbytes # This will provide detailed information about your CUDA setup and might reveal any potential issues.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/

In [ ]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/path/to/cuda/lib64

In [ ]:
!nvidia-smi # verify CUDA installation

# reinstall bitsandbytes
!pip install bitsandbytes --force-reinstall

# If necessary, set LD_LIBRARY_PATH
# Replace /path/to/cuda/lib64 with the actual path to your CUDA libraries
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/path/to/cuda/lib64

Mon Feb 10 15:37:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#import libraries
import os
import gc
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
#define model, dataset and parameters
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "yahma/alpaca-cleaned"
new_model = "Llama-2-7b-finetuned-alpaca"

# params for loRA (Low -Rank Adaptaiton)
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# define to use 4-bit quantization with bitsandbytes (giup giam trong luong cho model)
use_4bit = True
bnb_4bit_compute_dtype = "float16" #type of data calculate
bnb_4bit_quant_type = "nf4" #kind of quantization
use_nested_quant = False  # do not use nested (double quantization)

# convert float16 -> torch.float16
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

#create bitsandbytes with imported params
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

#set device_map
device_map = {"": 0}

#check GPU support bfloat16
if compute_dtype == torch.float16 and use_4bit:
  major, _ = torch.cuda.get_device_capability()
  if major >= 8:
    print("=" * 80)
    print("your GPU supports bfloat16: accelerate training with bf16=True")
    print("=" * 80)

In [ ]:
#load root model from Hungging Face with defined quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False # off cache to avoid erros
model.config.pretraining_tp = 1 # lien quan den tensor parallelism

# down tokenizer similer with model
tokenier = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

#set token padding = token eos ( de dam baon tinh nhat quan)
tokenier.pad_token = tokenier.eos_token

#set padding on right to avoid errors
tokenier.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
#load and check dataset
dataset = load_dataset(dataset_name, split="train")

#get 1000 sample
dataset = dataset.select(range(1000))

#print
for i, example in enumerate(dataset):
  print(f"sample {i +1}")
  print(f"Instruction: ", example["instruction"])
  if example["input"]:
    print(f"input: ", example['input'])
  print(f"output: ", example["output"])
  print("--------------------------\n")
  if i >= 4: # print 5 samples to check
    break

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

sample 1
Instruction:  Give three tips for staying healthy.
output:  1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.

2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.

3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.
--------------------------

sample 2
Instruction:  What are the three primary colors?
output:  The three primary colors are red, blue, and yellow. These colors are called prima

In [ ]:
# define formatting func
def formatting_func(example):
  """
  nhận vào 1 mẫu dữ liệu (dictionary) và chuyển đổi nó thành chuỗi theo định dạng: [INST] instruction + (nếu có input: "Input: input") [/INST] output
  """
  output_texts = []
  # if input xem có rỗng ko
  additional_input = ""
  # Check if 'input' is a list and join its elements if so
  input_value = example.get("input", "")
  if isinstance(input_value, list):
    input_value = " ".join(input_value)
  if input_value.strip() != "":  # Now use strip() on the string
    additional_input = f"input: {input_value}"
  # ghep instruction with input neu co vao trong [INST] [/INST], sau do noi voi output
  text = f"[INST] {example['instruction']} {additional_input} [/INST] {example['output']}"
  output_texts.append(text)
  return output_texts

In [ ]:
#define trainingArguments
output_dir = "./results"
num_epochs =1
per_device = 2
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 0
logging_steps = 25
learning_rate = 2e-4
weight_decay = 0.001
fp16 = False
bf16 = False
max_grad_norm = 0.3
warmup_ratio = 0.03
group_by_length = True
lr_scheduler_type = "cosine"
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=per_device,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    logging_steps=logging_steps,
    save_steps=save_steps,
    report_to="tensorboard"
)

max_seq_length = None
packing = False

In [ ]:
#create SFTTrainer and training
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
    ),
    formatting_func=formatting_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenier,
    args=training_arguments,
    packing=packing
)

#training
trainer.train()

#after training, save model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


In [ ]:
#eval model
prompt = "Giải thích chi tiết cách thức hoạt động của kiến trúc Transformer trong học sâu, bao gồm cơ chế self-attention, cách thức hoạt động của encoder và decoder cũng như vai trò của positional encoding"

#tạo pipeline cho task sinh văn bản, đặt max_length để giới hạn đầu ra
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenier, max_length=400)

result = pipe(f"[INST] {prompt} [INST]")
print(result[0]["generated_text"])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] Giải thích chi tiết cách thức hoạt động của kiến trúc Transformer trong học sâu, bao gồm cơ chế self-attention, cách thức hoạt động của encoder và decoder cũng như vai trò của positional encoding [INST]  Sure, I'd be happy to explain the Transformer architecture in more detail!
 nobody

The Transformer architecture was introduced in a paper by Vaswani et al. in 2017 and has since become a widely-used and influential model in the field of natural language processing (NLP). The Transformer is a type of neural network designed primarily for sequence-to-sequence tasks, such as machine translation, text summarization, and text generation.

At its core, the Transformer architecture is based on the idea of self-attention, which allows the model to attend to different parts of the input sequence simultaneously and weigh their importance. This is in contrast to traditional recurrent neural networks (RNNs), which process the input sequence one step at a time and have recurrence connection

In [ ]:
# --------------------- Cell 9: Merge LoRA Weights & Push Model to Hugging Face Hub ---------------------

# Bước 1: Giải phóng bộ nhớ từ các cell trước
# Xóa các biến không cần thiết và gọi garbage collection, sau đó xóa cache của CUDA
# Only delete model and pipe if they exist
if 'model' in globals(): del model
if 'pipe' in globals(): del pipe
# trainer is already deleted after training is complete

import gc
gc.collect()
torch.cuda.empty_cache()  # Xóa cache bộ nhớ GPU

# Nếu bạn vẫn gặp lỗi out-of-memory, hãy tải mô hình trên CPU.
# Dùng device_map="cpu" để tránh việc load toàn bộ mô hình lên GPU.
print("Loading base model onto CPU to avoid out-of-memory errors...")

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,  # Giúp giảm bộ nhớ CPU sử dụng khi load mô hình
    return_dict=True,
    torch_dtype=torch.float16,  # Hoặc torch.bfloat16 nếu GPU hỗ trợ
    device_map="cpu"  # Load mô hình trên CPU để tránh lỗi OOM
)

# Bước 2: Merge trọng số LoRA vào mô hình gốc và unload module PEFT
print("Merging LoRA weights...")
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Bước 3: Tải lại tokenizer từ mô hình gốc để đảm bảo định dạng token nhất quán
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Bước 4: Thiết lập locale cho lệnh login (đảm bảo sử dụng UTF-8)
import os
os.environ["LC_ALL"] = "en_US.UTF-8"
os.environ["LANG"] = "en_US.UTF-8"

# Thay thế bằng cách sử dụng `huggingface_hub` library để login:
from huggingface_hub import login
# Nhập token Hugging Face của bạn ở đây
# Bạn có thể lấy token từ trang web Hugging Face trong phần Settings -> Access Tokens
# **REPLACE "YOUR_ACTUAL_TOKEN" WITH YOUR REAL HUGGING FACE TOKEN**
YOUR_HUGGINGFACE_TOKEN = "NONE"  # Replace with your actual token
login(token=YOUR_HUGGINGFACE_TOKEN)

# Bước 5: Đẩy mô hình và tokenizer đã fine-tuned lên Hugging Face Hub
print("Pushing model and tokenizer to Hugging Face Hub...")

# **Change 1: Modify the repository ID to include your username**
# Replace "your-username" with your actual Hugging Face username
new_model_repo = f"vinhduong0107zzz/{new_model}"

model.push_to_hub(new_model_repo, use_temp_dir=False)
tokenizer.push_to_hub(new_model_repo, use_temp_dir=False)

print("Done!")

Loading base model onto CPU to avoid out-of-memory errors...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Merging LoRA weights...
Pushing model and tokenizer to Hugging Face Hub...


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Done!
